In [4]:
from utils.complete import create


import inspect

def select_and_call_function(user_input: str, function_list: list):
    prompts = []
    for function in function_list:
        source = inspect.getsource(function)
        docstring = function.__doc__ if function.__doc__ else "No docstring provided."
        prompts.append(f"Function: {function.__name__}\nDocstring: {docstring.strip()}\nSource:\n{source}")

    combined_prompt = "\n\n".join(prompts) + f"\n\nUser input: {user_input}\nWhich function should be called?"
    selected_function_name = interpret_with_openai(combined_prompt)

    # Find and call the selected function
    for function in function_list:
        if function.__name__ == selected_function_name:
            return function()

    raise ValueError("No suitable function found.")


def interpret_with_openai(prompt: str) -> str:
    """
    Uses OpenAI to interpret which function to call based on the given prompt.

    Args:
    prompt (str): The prompt to send to OpenAI.

    Returns:
    str: The name of the function to call.
    """
    print(prompt)
    response = create(prompt=prompt)  # Assuming 'create' is a wrapper for OpenAI API call
    print(response)
    return response.strip()

def function_a():
    """This function does something related to task A."""
    # Function implementation
    print("Function A called")

def function_b():
    """This function handles task B."""
    # Function implementation
    print("Function B called")

functions = [function_a, function_b]
user_input = "I need to handle task B"

# Call the function selector
select_and_call_function(user_input, functions)


Function: function_a
Docstring: This function does something related to task A.
Source:
def function_a():
    """This function does something related to task A."""
    # Function implementation
    print("Function A called")


Function: function_b
Docstring: This function handles task B.
Source:
def function_b():
    """This function handles task B."""
    # Function implementation
    print("Function B called")


User input: I need to handle task B
Which function should be called?
function_b
Function B called


In [2]:
import calendar
import datetime as dt

def get_this_month():
    # Get the current date to determine the current month and year
    today = dt.date.today()
    current_year = today.year
    current_month = today.month

    # Create a TextCalendar object starting from Sunday (you can change it to MONDAY if you prefer)
    cal = calendar.TextCalendar(calendar.SUNDAY)

    # Print the calendar for the current month
    cal_text = cal.formatmonth(current_year, current_month)


    return f"Today: {today}\n{cal_text}"

get_this_month()

'Today: 2023-11-20\n   November 2023\nSu Mo Tu We Th Fr Sa\n          1  2  3  4\n 5  6  7  8  9 10 11\n12 13 14 15 16 17 18\n19 20 21 22 23 24 25\n26 27 28 29 30\n'

In [27]:
from dataclasses import dataclass, field
from utils.complete import create

from datetime import datetime
from typing import Optional, List, Any


@dataclass
class VEvent:
    id: Optional[Any] = field(default=None)
    dtstart: datetime = None
    dtend: Optional[datetime] = None
    duration: Optional[str] = None
    summary: Optional[str] = None
    description: Optional[str] = None
    location: Optional[str] = None


create(prompt=f"{get_this_month()}\nYou are a icalendar expert assistant. I want to have a sync meeting with my team tomorrow morning\n```vevent\nBEGIN:VEVENT")

'DTSTART:20231121T090000\nDTEND:20231121T100000\nSUMMARY:Sync Meeting with Team\nEND:VEVENT'

In [47]:
event = get_vevent("Sync with the team tomorrow morning")
event




VEvent(id='5dc72aae-2c2c-4fcd-acc9-04f6fccc3c05', dtstart='20231121T090000', dtend='20231121T100000', duration=None, summary='Sync with the team', description=None, location=None)

In [46]:
import json
import uuid

import openai
from dateutil import parser
from icontract import require, ensure
from sqlmodel import Field, SQLModel, Relationship, select
from datetime import datetime
from typing import Optional, List

from shipit.cli import get_session, get_mem_store
from utils.crud_tools import add_model, update_model, delete_model, get_model


def get_vevent(prompt):
    vevent = create(prompt=f"You are a icalendar expert assistant. You specialize in creating VEvents. You are very careful to make sure the start and end times are perfect.\n{prompt}\n{get_this_month()}\nn\```vevent\nBEGIN:VEVENT", stop="\nEND:VEVENT")

    # 'DTSTART:20231123T160000\nDTEND:20231123T170000\nSUMMARY:Holiday dinner with the family\nDESCRIPTION:Thanksgiving Day at 4pm\nEND:VEVENT'
    # I need to turn this in kwargs for Event using a dict comp
    kwargs = {line.split(":")[0].lower(): line.split(":")[1] for line in vevent.split("\n") if line != "END:VEVENT"}

    event = VEvent(**kwargs)
    event.id = str(uuid.uuid4())

    return event



In [43]:
def ical_prompt(prompt):
    return f"You are a icalendar expert assistant.\n{prompt}\n"



needed_fields = [field for field in event.__dict__.keys() if event.__dict__[field] is None]

missing = ical_prompt(f"What information do you need to fill this event? {event}\nMissing fields: {needed_fields}\n\nAssistant: This is what I recommend to fill the fields with:")
create(prompt=missing)

'- Duration: 1 hour\n- Description: Discuss current projects and tasks with the team\n- Location: Virtual meeting via Zoom'

In [49]:
from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import List, Optional


# Define the start and end times for the day
start_time = datetime(2023, 11, 21, 9, 0)  # 9:00 AM
end_time = datetime(2023, 11, 21, 17, 0)  # 5:00 PM

# Create a list of events for the day
events = [
    VEvent(id='1', dtstart=start_time, dtend=start_time + timedelta(hours=1), summary='Morning Standup Meeting'),
    VEvent(id='2', dtstart=start_time + timedelta(hours=2), dtend=start_time + timedelta(hours=3), summary='Code Review Session'),
    VEvent(id='3', dtstart=start_time + timedelta(hours=4), dtend=start_time + timedelta(hours=5), summary='Development Workshop'),
    VEvent(id='4', dtstart=start_time + timedelta(hours=2, minutes=30), dtend=start_time + timedelta(hours=4), summary='Lunch Break'),
    VEvent(id='5', dtstart=start_time + timedelta(hours=3, minutes=30), dtend=start_time + timedelta(hours=4, minutes=30), summary='Overlap Meeting'),
]

# Print the events
for event in events:
    print(event)


create(prompt=f"{get_this_month()}\nYou are a icalendar expert assistant. which meetings overlap?\n```vevent\nBEGIN:VEVENT")


VEvent(id='1', dtstart=datetime.datetime(2023, 11, 21, 9, 0), dtend=datetime.datetime(2023, 11, 21, 10, 0), duration=None, summary='Morning Standup Meeting', description=None, location=None)
VEvent(id='2', dtstart=datetime.datetime(2023, 11, 21, 11, 0), dtend=datetime.datetime(2023, 11, 21, 12, 0), duration=None, summary='Code Review Session', description=None, location=None)
VEvent(id='3', dtstart=datetime.datetime(2023, 11, 21, 13, 0), dtend=datetime.datetime(2023, 11, 21, 14, 0), duration=None, summary='Development Workshop', description=None, location=None)
VEvent(id='4', dtstart=datetime.datetime(2023, 11, 21, 11, 30), dtend=datetime.datetime(2023, 11, 21, 13, 0), duration=None, summary='Lunch Break', description=None, location=None)
VEvent(id='5', dtstart=datetime.datetime(2023, 11, 21, 12, 30), dtend=datetime.datetime(2023, 11, 21, 13, 30), duration=None, summary='Overlap Meeting', description=None, location=None)


'DTSTART:20231120T090000\nDTEND:20231120T100000\nSUMMARY:Meeting 1\nEND:VEVENT\nBEGIN:VEVENT\nDTSTART:20231120T093000\nDTEND:20231120T103000\nSUMMARY:Meeting 2\nEND:VEVENT\n```\n\nThe two meetings overlap as they both occur on November 20, 2023 from 9:30 AM to 10:00 AM.'